In [12]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================
# -- Copied from lecture
%load_ext autoreload
%config IPCompleter.greedy=True
%autoreload 1
%aimport util
from util import util
import numpy as np4
# from matplotlib import pyplot as plt
import pandas as pd
from sklearn.neighbors import KernelDensity
from IPython.display import display

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 4)

data_folder = '../resources/dataset'
file_name = '1_gecco2019_water_quality.csv'
# Load the input data
data_path = f'{data_folder}/{file_name}'
data = pd.read_csv(data_path)
data['Time'] = pd.to_datetime(data['Time'])
data.set_index('Time', inplace=True)
data = data.drop(columns=["Unnamed: 0"]) # The index was stored as an unnamed column

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Investigation

In [38]:
print("Columns: ")
{column: data[column].dtype for column in data.columns}

Columns: 


{'Tp': dtype('float64'),
 'pH': dtype('float64'),
 'Cond': dtype('float64'),
 'Turb': dtype('float64'),
 'SAC': dtype('float64'),
 'PFM': dtype('float64'),
 'Event': dtype('bool')}

In [43]:
display((data.index[1:] - data.index[:-1]).value_counts())
print("\nTherefore, there are no entire rows missing.")
print("However, all columns themselves except for the label contain missing values: \n")
data.isna().any()

Time
0 days 00:01:00    132479
Name: count, dtype: int64


Therefore, there are no entire rows missing.
However, all columns themselves except for the label contain missing values: 



Tp        True
pH        True
Cond      True
Turb      True
SAC       True
PFM       True
Event    False
dtype: bool

## Auto Correlation Plot

In [29]:
data.isna().any()

Tp        True
pH        True
Cond      True
Turb      True
SAC       True
PFM       True
Event    False
dtype: bool

In [25]:
#üfor column in data.columns:
##    display(util.plot_autocorrelation(data[column], figsize))
util.plot_autocorrelation(data["Tp"], figsize)


KeyError: 'value'

<Figure size 900x300 with 0 Axes>

## Test for markov property

In the lectures, wen defined the alarm signal we strive to minimize:
$$
-\log f(x, θ) \geq ε
$$
where $$ f(x, θ) $$ is the true distribution function.

## Investigation
Determining the period:

# Multivariate Kernel Density Estimation
The first approach presented in the lecture is **Kernel Density Estimation**

In order to employ **KDE**, we need to determine the optimal **Kernel Function** and **Bandwidth**. 
Since we have multiple columns, we cannot use the Rule Of Thumb for the latter. Therefore, we need to optimize the following term according to the lecture: 
$$
\mathop{\arg\max}_{h} \mathbb{E}_{x \sim f(x), \bar{x} \sim f(x)}\left[ L(h, x, \bar{x})\right]
$$
where
- $$
L(h, x, \bar{x}) = \prod_{i=1}^m \hat{f}(x_i, \bar{x}_i, h)
$$
- $\hat{f}$ is the density estimator (which outputs a probability)
- $\bar{x}$ the training set

according to the lecture.
